In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
url = 'https://www.vivareal.com.br/aluguel/rj/rio-de-janeiro/apartamento_residencial/'

In [5]:
paginas = [f'?pagina={i}' for i in range(2,10)]
paginas.insert(0,'')

In [6]:
def get_info(url):
    response = requests.get(url).content
    soup = BeautifulSoup(response)
    
    aluguel_lista = [p.text.strip() for p in 
                     soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'})]
    
    quartos_lista = [p.text.strip() for p in 
                    soup.find_all('li', {'class':'property-card__detail-item property-card__detail-room js-property-detail-rooms'})]

    banheiro_lista = [p.text.strip() for p in 
                      soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'})]

    vaga_lista = [p.text.strip() for p in 
                  soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-garage js-property-detail-garages'})]

    area_lista = [p.text.strip() for p in 
                 soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-area'})]
 
    df = pd.DataFrame({
            'aluguel': aluguel_lista,
            'quartos': quartos_lista,
            'banheiros':banheiro_lista,
            'vagas':vaga_lista,
            'area':area_lista
        })
    
    return df

In [7]:
df = pd.DataFrame([], columns=['aluguel','quartos', 'banheiros','vagas','area'])

for pg in paginas:
    df = pd.concat([df,get_info(f'{url}{pg}')])

In [8]:
df.head()

,aluguel,quartos,banheiros,vagas,area
0,R$ 3.900 /Mês,3 Quartos,3 Banheiros,1 Vaga,100 m²
1,R$ 3.200 /Mês,3 Quartos,3 Banheiros,2 Vagas,115 m²
2,R$ 1.550 /Mês,1 Quarto,2 Banheiros,1 Vaga,65 m²
3,R$ 1.500 /Mês,1 Quarto,2 Banheiros,-- Vaga,45 m²
4,R$ 2.800 /Mês,3 Quartos,3 Banheiros,2 Vagas,120 m²


In [9]:
df.shape

(324, 5)

In [10]:
df = df.drop_duplicates(subset=['aluguel','quartos','banheiros','vagas','area'], keep='first')

In [12]:
df.shape

(36, 5)

In [13]:
df.to_excel('calculadora.xlsx')